<a href="https://colab.research.google.com/github/siyandaV-0/llm-playground/blob/main/meeting_minutes_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meeting minutes creator

In this colab, we make a meeting minutes program.

It includes useful code to connect your Google Drive to your colab.

Upload your own audio to make this work!!

https://colab.research.google.com/drive/1KSMxOCprsl1QRpt_Rq0UqCAyMtPqDQYx?usp=sharing

This should run nicely on a low-cost or free T4 box.

In [9]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [10]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [11]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [12]:
# New capability - connect this Colab to my Google Drive

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/Siyanda AI Research/Gen AI Research/LLMS/LLM Engineering: Master AI & Larg Language Models(LLMs)/llm_engineering/week3/videoplayback.mp4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Audio File

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

In [13]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [15]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [16]:
# Use the Whisper OpenAI model to convert the Audio to Text

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

I shall now call upon the President to address the Joint Secretary. Speaker of the National Assembly, Ms. Thandi Mudise, Chairperson of the National Council of Provinces. On the point of order, Speaker. On the point of order, Speaker. What's the point of order? Can the President please sit down? I'm following the rules, Speaker. What's the point of order? Can we follow the rules? What's the point of order? Can we follow the rules? According to Rule 14C, read alongside 14S. If you read them now, you will realize that the President must sit down before I raise the point of order. Please sit down, President. We're going to help you run this Parliament. Sit down, President. Follow the rules. We're going to help you. You want to follow the rules? Follow the rules and sit down. Can you find out what is the point of order? Well, the point of order, Speaker, is that we cannot sit here in this Parliament with millions of South Africans watching us when they know that they are in darkness becaus

In [20]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Speech given by Julius Malema in the 2020 SONA debat. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [21]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [22]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Speech given by Julius Malema in the 2020 SONA debat. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
I shall now call upon the President to address the Joint Secretary. Speaker of the National Assembly, Ms. Thandi Mudise, Chairperson of the National Council of Provinces. On the point of order, Speaker. On the point of order, Speaker. What's the point of order? Can the President please sit down? I'm following the rules, Speaker. What's the point of order? Can we follow the rules? What's the point of order? Can we follow

In [23]:
response = tokenizer.decode(outputs[0])

In [24]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Speech given by Julius Malema in the 2020 SONA debat. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
I shall now call upon the President to address the Joint Secretary. Speaker of the National Assembly, Ms. Thandi Mudise, Chairperson of the National Council of Provinces. On the point of order, Speaker. On the point of order, Speaker. What's the point of order? Can the President please sit down? I'm following the rules, Speaker. What's the point of order? Can we follow the rules? What's the point of order? Can we follow the rules? According to Rule 14C, read alongside 14S. If you read them now, you will realize that the President must sit down before I raise the point of order. Please sit down, President. We're going to help you run this Parliament. Sit down, President. Follow the rules. We're going to help you. You want to follow the rules? Follow the rules and sit down. Can you find out what is the point of order? Well, the point of order, Speaker, is that we cannot sit here in this Parliament with millions of South Africans watching us when they know that they are in darkness because of one man who has made sure that our state-owned entities are in a state of collapse. Today, Westcom cannot keep light in our houses. Today, we cannot fly to Deben because there is a minister who is making sure that SAA is dysfunctional. Today, we sit here with a minister that is working for white monopoly genital making sure that the state-owned entities go to his friends. We call for an adjournment. The President must go outside and make sure that he makes an announcement to the nation that Pravin Godan is no longer a minister of this country. We cannot sit with him. We want to sit with Pravin Godan, not here. Pravin Godan must go. Pravin Godan must go. Please. That's not the point of order. Point of order. With the greatest respect, Madam Speaker, the Constitution is also on our side. Because we are members sworn to the Constitution, we also have a right to partake in these proceedings. I just want to really bring to your attention Can I respond to you on that point of order? I will be done in a moment, Speaker. I'll be done. We can't gather like things are normal here when things are abnormal. Pravin Godan, Mr. Selurama Posa, misled you. He misled you together with another man and you fired the other man and not Pravin Godan. Take your seat. Please fire Pravin Godan now so that we can go. Take your seat. No honourable members. EFF, if they want to engage you, let's go outside and engage outside. Rekan Magitaga, let's go and engage outside. Let's go, let's go, let's go. Order. Let's go, let's go. Honourable members. Honourable members. Take your seats. Honourable members. Take your seats. No. Honourable members. Take your seats. Honourable members. Honourable members. Honourable members. Take your seats. You all have a minute to take your seat or march out. Take your seats. Honourable Shibambu, take your seat. Honourable members. Honourable members. Honourable members. Honourable members, you clearly do not have any interest in the business of this house today. You are delaying us. You are refusing us to start with the business of the day. Honourable members. There is, there is, there is an input on the floor. Any member who feels they do not want to participate in the proceedings of this joint sitting today, you are free to walk out and allow us and the rest of this country to enjoy the state of the nation. Thank you. If you choose to sit and do not leave, you will, you will sit and allow us to continue with the decorum of this house so that we can get on with the business of the day. Any other member who will stand up in this house and disrupt, please leave in peace. Any member, you are disrupting. No. Honourable Shibambu, what do you understand to be a disruption? A disruption is when you do not use the rules to make interruptions. An interruption is allowed in terms of the rules that are contained in this book which guides all of us here now. There are two issues which you cannot overlook, Speaker. One, you have got a semi-literate person who can't think and say, we must go and fight outside. No, I'm not entertaining that. That is one. And number two, if we cannot find each other here in terms of this, which I propose because we do not have a chief who is going to miss the day, the whips must go outside to come with a solution. So that we can guide these whips on what must happen. That is a concrete proposal so that we come to guide these house as whips of what should happen. That is the concrete proposal. Honourable Shibambu, you sit and you let us proceed with the business of today. No, Honourable Malemba, we have had enough. No, no, on a point of order. You keep on saying members of Parliament are disrupting Parliament. When we stand up and speak through you, you cannot call that a disruption. You cannot call our inputs here a disruption. We've got a right to be here. We've got a right to speak here. And that's what we are getting paid for. And we can't, when we execute our responsibility, say we don't have time. We've got all the time. We are getting paid to be here. The chief whip has made a progressive proposal that lets us adjourn for five minutes so that the chief whips can talk outside. Since there was no chief whip forum. Honourable Malemba, please take your seat. No. Please take your seat. There was no chief whips forum. Honourable Malemba, take your seat. There is nothing we can do. Honourable Malemba, take your seat. Which rules are you quoting? I am saying please take your seat. But guide us through the rules. If you do not wish to take your seat, sir, please leave this house. Honourable Mkalipi, I am not taking any further points of orders. You are simply retarding the progress of this house tonight. Honourable Godri, I am not taking your point of order. You are simply retarding our progress. Take your seat. Take your seat. If you do not wish to take your seat, I am going to ask that you be held out of this house. Please, just before that. No. Take your seat. No, take your seat. No, Honourable Malemba. You cannot come here and take away the rights of the rest of this house. This one has taken the rights of our people to have life and to fight to the death. I will not have dialogue with you. Take your seat. But, Madam Speaker. Take your seat. Take your seat. It's not speedless. Take your seat. That Honourable Malemba, no member has any right to threaten any member. Neither does any member have the right to take away and to disadvantage any other member in any other fashion. So, you are not angels because you infringe on the rights of all the other members in this house to proceed. And that is why I am saying I am not taking any point. I would like to make a plea one more time. That we should allow the President to proceed with his State of the Nation address. Honourable President, please proceed. Honourable Chairperson of the NCOP, on a point of order. I have not recognised you. Honourable Malemba. Honourable Malemba. Honourable Malemba, I have not recognised you, so please sit down. Please proceed, Honourable President. Speaker. Speaker. Can you please recognise us? I want us to work according to rules here. Honourable Malemba. Honourable Malemba, we have not been recognised. Please sit down. Please sit down. Please sit. Please sit. Honourable President. No, Brahambi, you are making a mistake. You are making a mistake. You are making a mistake. Recognise me. I want to speak according to the rules. Honourable Speaker. You ought to say to us, according to which rule are you no longer going to take hands on point of order? Number one. Number two. There is no one who is going to be treated as a party here. There is no party here. They are members of Parliament who shall be recognised as individual members of Parliament. So there is no party. So please, we and there is no majority and there is no minority. Honourable Malemba. If you are going to go with that narrative of minority, minority will know us. Honourable Malemba. Honourable President. Honourable President. We are now proceeding, Honourable Members, with the State of the Nation Address. Speaker of the National Assembly. Mr. Andi Mudise. The Chairperson of the National Council of Provinces. Mr. Amos Masondo. Speaker of the National Assembly. A wnaiff y Prif Weinidog ddatganiad am y peth y mae'r Prif Weinidog wedi'i wneud? Y Llywodraeth. Rydyn ni'n mynd i'r cyfrifiadau o'r ysgol hwn. Bydd y golygau'n mynd i'r ysgol hwn. Mae'r ysgol hwn yn cael eu cyfrifiadu. Y Llywodraeth! Y Llywodraeth! Y Llywodraeth! Y Llywodraeth! Y Llywodraeth! Y Llywodraeth! Y Llywodraeth! Y Llywodraeth!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Meeting Minutes**
====================

**Attendees:**
- Julius Malema (EFF Member of Parliament)
- Thandi Mudise (Speaker of the National Assembly)
- Amos Masondo (Chairperson of the National Council of Provinces)
- Other Members of Parliament

**Location:** Parliament of South Africa
**Date:** 2020

**Summary:**
The meeting was a debate on the State of the Nation Address (SONA) in the South African Parliament. Julius Malema, a member of the Economic Freedom Fighters (EFF), led a protest against the President, calling for the removal of Pravin Gordhan as a minister. The debate turned into a dispute between Malema and the Speaker of the National Assembly, with Malema accusing the Speaker of not following the rules and the Speaker asking Malema to take his seat.

**Discussion Points:**

* Julius Malema's protest against the President and the Speaker
* Malema's call for the removal of Pravin Gordhan as a minister
* The Speaker's attempts to maintain order and follow the rules
* Malema's accusations against the Speaker of not recognizing him and not following the rules
* The debate turned into a dispute between Malema and the Speaker

**Takeaways:**

* The debate on the SONA was disrupted by a protest led by Julius Malema
* Malema's protest was against the President and the Speaker, and he called for the removal of Pravin Gordhan as a minister
* The Speaker attempted to maintain order and follow the rules, but the debate turned into a dispute between Malema and the Speaker
* Malema accused the Speaker of not recognizing him and not following the rules

**Action Items:**

* **Owner:** Speaker of the National Assembly
* **Action:** Recognize Julius Malema and other members of Parliament according to the rules
* **Status:** Not completed
* **Owner:** Julius Malema
* **Action:** Refrain from disrupting the debate and follow the rules
* **Status:** Not completed
* **Owner:** Chief Whips
* **Action:** Meet outside to discuss and find a solution to the dispute
* **Status:** Not completed

**Next Steps:**
The debate on the SONA was eventually allowed to proceed, but the dispute between Julius Malema and the Speaker remained unresolved. The Chief Whips were asked to meet outside to discuss and find a solution to the dispute.<|eot_id|>